In [3]:
import os
from pathlib import Path
os.chdir(Path().resolve().parent)
os.getcwd()

'/nvme/wangding/Desktop/tsim/tsim'

In [4]:
import matplotlib.pyplot as plt
import networkx as nx
import imageio
from pathlib import Path
from graph.load import load_graph
from eval import model_struc_to_eval, get_real_feat, get_sim_feat, get_struc_df, get_cf_veh_pair, raw_struc_to_eval, compare_struc,model_feat_to_eval,model_struc_to_eval
from graph.process import generate_unique_node_id
import pandas as pd
import os
#import cv2
import pickle
import torch
from matplotlib.pyplot import figure
import math
import numpy as np
plt.style.use('default')
from sklearn.preprocessing import MinMaxScaler
import re
import numpy as np

import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import numpy as np
import re

import glob
import pickle
from collections import defaultdict

### 1. load results with different time interval

In [14]:
exp_dir = Path().resolve().parent / "experiment"

In [15]:
exp_dir

PosixPath('/nvme/wangding/Desktop/tsim/experiment')

In [17]:
def load_sim_feat(out_dir,file_name):    
    with open(out_dir / file_name, "rb") as g:
        model_graph = pickle.load(g)
    unique_key = list(model_graph.keys())[0]
    sim_struc, sim_feat = model_graph[unique_key]
    return sim_feat

def load_data(case,test,out,training_step,pred_step):
    exp_setting = exp_dir / case
    train_data_dir = exp_setting   / "data" / test / "train_data"
    real_data_dir = exp_setting / "data" / test / "test_data"
    out_dir = exp_setting / "data" / test / out
    node_all = pd.read_csv(train_data_dir / "node_all.csv")
    node_id_dict = generate_unique_node_id(node_all)
    
    file_list = os.listdir(out_dir)
    file_list = [file_name for file_name in file_list if file_name.endswith(".p")]
    
    sim_feat0 = load_sim_feat(out_dir,file_list[0])
    sim_feat1 = load_sim_feat(out_dir,file_list[1])
    sim_feat2 = load_sim_feat(out_dir,file_list[2])
    sim_feat3 = load_sim_feat(out_dir,file_list[3])
    sim_feat4 = load_sim_feat(out_dir,file_list[4])
    sim_feat5 = load_sim_feat(out_dir,file_list[5])
    sim_feat6 = load_sim_feat(out_dir,file_list[6])
    sim_feat7 = load_sim_feat(out_dir,file_list[7])
    sim_feat8 = load_sim_feat(out_dir,file_list[8])
    sim_feat9 = load_sim_feat(out_dir,file_list[9])
    
    sub_feat = [sim_feat0, sim_feat1, sim_feat2, sim_feat3, sim_feat4, sim_feat5, sim_feat6, sim_feat7, sim_feat8, sim_feat9]

    for i, sim_feat in enumerate(sub_feat):
        time_start = training_step + pred_step * (i)
        time_end = time_start + pred_step
        for node_type in sim_feat:
            for node_id in sim_feat[node_type]:
                for feat_name in sim_feat[node_type][node_id]:
                    if feat_name == "time":
                        sim_feat[node_type][node_id][feat_name] = torch.arange(time_start, time_start + len(sim_feat[node_type][node_id][feat_name]))
                        
    # sub_feat =[sim_feat0,sim_feat1,sim_feat2,sim_feat3,sim_feat4]
    n = len(sub_feat)
    #sim_feat_list = [sim_feat1,sim_feat2,sim_feat3,sim_feat4] 
    combined_feat = defaultdict(dict)
    for node_type in set([key for sim_feat in sub_feat for key in sim_feat.keys()]):
        for node_id in set([key for sim_feat in sub_feat for key in sim_feat.get(node_type, {}).keys()]):
            combined_feat[node_type][node_id] = {}
            for feat_name in set([key for sim_feat in sub_feat for key in sim_feat.get(node_type, {}).get(node_id, {}).keys()]):
                feat_list = []
                for sim_feat in sub_feat:
                    feat_list.append(sim_feat.get(node_type, {}).get(node_id, {}).get(feat_name, torch.empty(0)))
                feat_value = torch.cat(feat_list, dim=0).squeeze()
                combined_feat[node_type][node_id][feat_name] = feat_value
    sim_feat = combined_feat
    return sim_feat

In [20]:
training_step = 400
pred_step = 10
case =  "traci_tls"
test = "test2"
out = "out_dim_200_n_heads_4_n_layer_2_dr"

sim_feat = load_data(case,test,out,training_step,pred_step)

In [19]:
sim_feat['veh']

{0: {'length': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  'coor_y': tensor([0.5049, 0.5049, 0.5049, 0.5049, 0.5049, 0.5049, 0.5049, 0.5049, 0.5049,
          0.5049]),
  'time': tensor([400., 401., 402., 403., 404., 405., 406., 407., 408., 409.]),
  'speed': tensor([0.4587, 0.4851, 0.4810, 0.4846, 0.4508, 0.4607, 0.4714, 0.4738, 0.4624,
          0.4729]),
  'coor_x': tensor([0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004,
          0.0004]),
  'vmax': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  'pos_on_lane': tensor([0.0176, 0.0289, 0.0258, 0.0341, 0.0156, 0.0160, 0.0247, 0.0253, 0.0085,
          0.0238]),
  'acceleration': tensor([0.8865, 0.8881, 0.8745, 0.8743, 0.8962, 0.8934, 0.8850, 0.8801, 0.8875,
          0.8748]),
  'tlc_state': tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
          0.5000])},
 1: {'length': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  'coor_y': tensor([0.5017, 0.5017, 0.5017, 